0. ABSTRACT: 

We investigate ensemble learning methods, specifically random forests and boosted trees, to classify crime types utilizing data on incidents reported in Los Angeles County during the year 2020. The primary objective is to develop machine learning models capable of accurately predicting crime classifications based on various features associated with the reported incidents. Using data encompassing different attributes such as location, time, and other relevant factors, we aim to find relationships between these variables to enhance the classification accuracy of criminal activities. We seek to achieve an enhanced understanding of crime patterns, enabling law enforcement agencies to improve resource allocation and planning responses. The findings and insights derived from paper hold the potential to contribute to the field of crime analysis and facilitate more efficient measures for aiding victims within Los Angeles County and beyond. One other objective is seeking to understand the ideal location for police stations in the LA country based on the occurence and frequency of crime. We can compare this to the actual location of LA police stations to understand where the city could place future stations.

1. Introduction:

Los Angeles County has long been grappling with concerns surrounding its high crime rates. There is a wide variety of forms of criminal activities ranging from property crimes to violent offenses. Combatting this has been a consistent effort within the community and among law enforcement. While existing programs and resources offer support and aid to victims of many crimes, the efficiency of these could be greatly enhanced. Nationwide, many crime resources are in high demand, while others are going unused. The current system, though commendable, faces limitations in its ability to cater to the complex needs of crime victims. There exists a gap in the optimization of these programs, suggesting opportunities for improvement in their responses to distress calls and the following services. Enhancing these systems could better address the diverse needs and circumstances of victims, ensuring more effective aid and facilitating their path towards recovery and rehabilitation.

One way to improve these support systems is by analyzing crime data more systematically. By identifying trends and patterns in this data, it becomes possible to pinpoint areas with higher risks of certain crimes. This information can help in onsite assistance and emergency responses. It can also be used directing the county's resources, money, and services more strategically, making sure they're better suited to the specific needs of people and communities most impacted by crime. Using data from the crimes in LA county that were reported in 2020, we can create machine learning models to predict which areas of the county correlate to which crimes, assisting in the distribution of help resources. This also will help us understand where future police stations could be located. To analyze and understand these questions and potential we will first clean the data. We will also then look at some algorithms like kmeans, random forests, boosted forests and logistic regression to help us further understand crime classification. 

2. Data Explination and Cleaning

In order to do data cleaning we needed to important several packages. These were all imported in the usual way (numpy as np, pandas as pd, etc.)

To begin we had a dataset of observations on criminal reports from LA county that from 2020. This data set was rather large with over 82k data points. The data has 28 columns as well ranging from information about the time and location of the crime, a crime code (unique identifier of what the culprit was charged with), information about the victim, weapon used and more. Overall, the 28 columns represented different attributes that related to the crime that was reported. In order to avoid ethical issues and ensure that our models were not biased we first removed column attribute that had to with the culprit and defendant that could be unethical to use in classification predictions (race, gender and other similar columns).

Following this, due to the breadth of our data, we were able to drop rows with missing entries in critical columns that would help with classification. We took this approach as much of the data is nominal (like the description of the crime or weapon) making it difficult to replace missing values with the mean or other comparable approaches. After doing this we still had over 67k rows. In addition to entries with missing data, there were several crime types with no more than a few instances. With little data about such rarer offenses, we dropped any report involving a crime that happened less than 100 times. TODO: change number as needed. 

Depending on what algorithm we were using for future aspects of our report we may do additional data cleaning when necessary.

3. Ideal Police Station Locations

Much of our analysis revolves around being able to use components of a crime to predict what kind of a crime occurred. However, before we get to that stage, we first look at crime as a whole in LA. One question that we wanted to answer is to be able to understand the ideal locations for police stations in LA based on the frequency of crimes. This would help ensure that a police station was within a close distance to the areas with the most crime in LA. This would also help reduce the response time for first responders and could help improve safety, medical attention to victimes, and increase the odds of finding the culprit. 

In order to do this we wrote a kmeans class to help us identify the k ideal locations for a police station. The class is initialized with the k number of clusters, a maximum number of iterations, a norm, a tolerance check and normalization boolean. 

The class also consists of four main functions that are used to run the kmeans algorithm. This includes a fit, predict, fit_predict and plot function. The fit function computes the cluster centers from random intiial conditions.

In [ ]:
def fit(self, X, y=None):
        """Compute the cluster centers from random initial conditions.
        
        Parameters:
            X ((n_samples, n_classes) ndarray): the data to be clustered.
        """
        #set our centers and then normalize if specificed
        self.centers = X[np.random.choice(X.shape[0],self.n_clusters,replace=False)]
        if self.normalize == True:
            self.centers = np.reshape(self.centers/np.linalg.norm(self.centers,axis=1),(-1,1))
        for i in range(0,self.max_iter): #iterate thorugh max iter and create the label and new center
                label = np.argmin(np.linalg.norm(X[:,np.newaxis]-self.centers,ord=self.p,axis=2),axis=1)
                new_c = np.array([X[label==z].mean(axis=0) for z in range(self.n_clusters)])
                if np.linalg.norm(new_c-self.centers,ord=self.p) <self.tol: #if error is less than tol break
                     break
                self.centers = new_c #set the new center and normalize if specified
                if self.normalize == True:
                    self.centers = np.reshape(self.centers/np.linalg.norm(self.centers,axis=1),(-1,1))
        return self #return our object

This block does is a little dense so we will explain a little more beyond just the code comments. Essentially we choose random center locations for each of our k clusters. Then look at the distance (based on the specific norm) from each data point to our randomized centers. Each data point is labeled based on the smallest distance from it to all clusters. With each data point labeled we can now shift our centers to better fit the data. We also iterate through this process the number of times we set our max_iter attribute to. 

We will not go over the code for predict, fit_predict and plot for brevity sake. The functions classify each entry of the data on which cluster center it is closest to, return the labels and then plots the data and centers respectively. 

Now that we have our kmeans class we can start to examine the ideal locations for police stations. Based on data from LA county they would ideally have 21 police stations in the county. This gives us our ideal k for our kmeans algorithm. The following code shows what we did to get our kmeans graphs:

In [ ]:
location = my_data[['LON','LAT']] #get a dataframe with only longitude and latitude columns
new_loc = location[~(location == 0).all(axis=1)] #ensure that all entries are nonzero for both columns
new_data = new_loc[['LON','LAT']].values #create array with the longitude and latitude values
for x in [1,2,np.inf]: #iterate through our different norms 
    km = KMeans(n_clusters=21,p=x) #initialize our class, fit it with our data
    km.fit(new_data)
    y = km.predict(new_data) #predict our labels for the data and then use the plot function
    km.plot(new_data,y)

As part of the plot function, it will not only plot the ideal clusters for police stations but we also read in data to include the actual location of the current 17 police stations in LA county. We also utalized three different norms, the 1-norm, 2-norm and infitinity norm to get three different possible locations. This all yielded the following three graphs.

<image src="1_norm.png" controls>

This is the graph that corresponds to the 1-norm. 

<image src="2_norm.png" controls>

This is the graph that corresponds to the 2-norm.

<image src="three_norm.png" controls>

This is the graph that corresponds to the infinity norm.

Overall, each of these graphs have slightely different ideal locations for police stations. The black crosses correspond to the optimal location of police stations based on our crime data and the red crosses correspond to the actual location of police stations. We will not analyze the specific differences of the different results from each norm but wanted to provide potential ideal locations based on varying norms. However, there is much that we can learn from these three graphs overall. Each graph has at least three police stations in the north/west part of LA county (the range is from 3 to 5). Also, each graph shows that there would ideally be 1-2 police stations in the furthest south portion of LA county. Interestingly, the red crosses showcase that nearly all of the police stations in LA county are located  centerally in LA county. This showcases that LA could look into investing and placing more police stations in the north/west and south parts of its county. Overall, these kmeans maps help us to understand the location of crime and how to better place police stations to minimize the distance from crime to police stations. This has many practical applications as it could decrease response time and increase overall safety. 

Now that we have examined ideal locations, we will look into further classifying crime based on attributes of the crime. 

3. Feature Engineering

4. Visualizations and Analysis

5. Results and Conclusion

REFERENCES

Source for crime rates in LA over time

https://lasd.org/transparency/statistics/

Sources about need for victim resources. TO DO: do we want to include these?

https://time.com/5886815/crime-survivors-funding/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9837801/